# Классификация текстовых комментариев

***Цель исследования:*** Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нам нужно обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Значение метрики качества *F1* должно быть не меньше 0.75. 

***Ход исследования:*** Планируется 3 этапа:
1. Загрузка и подготовка данных.
2. Обучение моделей. 
3. Выводы.

**Описание данных:** Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [41]:
# Импортируем необходимые библиотеки и методы.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

## Подготовка

In [42]:
# Откроем и изучим датафрейм.
df = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [44]:
# Проверим датафрейм на пропуски.
df.isna().sum()

text     0
toxic    0
dtype: int64

In [45]:
# Посмотрим на распределение целевого признака.
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

**Промежуточный вывод:** Видим данные о тексте и токсичности комментариев. Данные соответствуют описанию задачи с правильными типами данных. Обзор данных не выявил пропуски.

In [46]:
# Очистим текст комментариев от разделителей и заглавных символов.
def clear_text(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", " ", text).strip()
    text = text.lower()
    return text

df['text'] = df['text'].apply(clear_text)
df.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m s...,0
2,hey man i m really not trying to edit war it...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember...,0


In [47]:
# Лемматизируем текст с помощью английской модели spaCy.
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

df['lemm'] = df['text'].apply(lambda x: lemmatize_text(x))
df.head()

,text,toxic,lemm
0,explanation why the edits made under my userna...,0,explanation why the edit make under my usernam...
1,d aww he matches this background colour i m s...,0,d aww he match this background colour I m se...
2,hey man i m really not trying to edit war it...,0,hey man I m really not try to edit war it ...
3,more i can t make any real suggestions on impr...,0,more I can t make any real suggestion on impro...
4,you sir are my hero any chance you remember...,0,you sir be my hero any chance you rememb...


**Промежуточный вывод:** Данные очищены от разделителей и заглавных символов, лемматизированы и готовы к работе.

## Обучение

### Подготовка данных к обучению

In [48]:
# Укажем стоп-слова.
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
RANDOM_STATE = 42
TEST_SIZE = 0.25
# Разделим данные на обучающую и тестовую выборки.
X = df['lemm']
y = df['toxic']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

In [51]:
# Векторизируем признаки.
tf_idf = TfidfVectorizer(stop_words=stopwords)

X_train_tf_idf = tf_idf.fit_transform(X_train)
X_test_tf_idf = tf_idf.transform(X_test)

### LogisticRegression

In [56]:
# Воспользуемся Pipeline и GridSearchCV для кросс-валидации модели.
pipe_lr = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('model', LogisticRegression(random_state=RANDOM_STATE, solver='liblinear', max_iter=200))])
param_lr = {'model__C':[1.0, 10.0]}
grid_lr = GridSearchCV(pipe_lr, param_lr, scoring = 'f1', cv=3, verbose=3, n_jobs=-1)
grid_lr.fit(X_train, y_train)

print('Лучшие параметры логистической регрессии:',grid_lr.best_params_)
print('F1 логистической регрессии:', grid_lr.best_score_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END ...................................model__C=1.0; total time=  13.6s
[CV 2/3] END ...................................model__C=1.0; total time=  12.8s
[CV 3/3] END ...................................model__C=1.0; total time=  13.5s
[CV 1/3] END ..................................model__C=10.0; total time=  19.4s
[CV 2/3] END ..................................model__C=10.0; total time=  17.2s
[CV 3/3] END ..................................model__C=10.0; total time=  16.9s
Лучшие параметры логистической регрессии: {'model__C': 10.0}
F1 логистической регрессии: 0.7750317855676528


**Промежуточный вывод:** Лучший параметр C логистической регрессии: 10. F1 = 0.775, что удовлетворяет заданному условию.

### DecisionTreeClassifier

In [13]:
# Инициализируем модель древа решений.
model_dt = DecisionTreeClassifier(random_state=RANDOM_STATE)

# Зададим параметры и воспользуемся GridSearchCV для определения лучших по метрике.
param_dt = {
    'max_depth':range (10, 21, 5)}
grid_dt = GridSearchCV(model_dt, param_dt, scoring = 'f1', cv=3, verbose=3, n_jobs=-1)
grid_dt.fit(X_train_tf_idf, y_train)

print('Лучшие параметры древа решений:',grid_dt.best_params_)
print('F1 древа решений:', grid_dt.best_score_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END ...................................max_depth=10; total time=  13.1s
[CV 2/3] END ...................................max_depth=10; total time=  13.0s
[CV 3/3] END ...................................max_depth=10; total time=  13.3s
[CV 1/3] END ...................................max_depth=15; total time=  14.7s
[CV 2/3] END ...................................max_depth=15; total time=  14.9s
[CV 3/3] END ...................................max_depth=15; total time=  14.8s
[CV 1/3] END ...................................max_depth=20; total time=  17.1s
[CV 2/3] END ...................................max_depth=20; total time=  16.9s
[CV 3/3] END ...................................max_depth=20; total time=  16.8s
Лучшие параметры древа решений: {'max_depth': 20}
F1 древа решений: 0.6434027291336637


**Промежуточный вывод:** Лучший параметр максимальной глубины древа решений: 20. F1 = 0.64, что не удовлетворяет заданному условию.

### RandomForestClassifier

In [14]:
# Инициализируем модель RandomForest.
model_rf = RandomForestClassifier(random_state=RANDOM_STATE, class_weight = 'balanced')

# Зададим параметры и воспользуемся GridSearchCV для определения лучших по метрике.
param_rf = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10]}
grid_rf = GridSearchCV(model_rf, param_rf, scoring = 'f1', cv=3, verbose=3, n_jobs=-1)
grid_rf.fit(X_train_tf_idf, y_train)

print('Лучшие параметры RandomForest:',grid_rf.best_params_)
print('F1 RandomForest:', grid_rf.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END ...................max_depth=5, n_estimators=50; total time=   4.6s
[CV 2/3] END ...................max_depth=5, n_estimators=50; total time=   4.6s
[CV 3/3] END ...................max_depth=5, n_estimators=50; total time=   4.5s
[CV 1/3] END ..................max_depth=5, n_estimators=100; total time=   9.1s
[CV 2/3] END ..................max_depth=5, n_estimators=100; total time=   9.0s
[CV 3/3] END ..................max_depth=5, n_estimators=100; total time=   9.0s
[CV 1/3] END ..................max_depth=10, n_estimators=50; total time=   8.9s
[CV 2/3] END ..................max_depth=10, n_estimators=50; total time=   8.7s
[CV 3/3] END ..................max_depth=10, n_estimators=50; total time=   8.8s
[CV 1/3] END .................max_depth=10, n_estimators=100; total time=  17.6s
[CV 2/3] END .................max_depth=10, n_estimators=100; total time=  17.4s
[CV 3/3] END .................max_depth=10, n_est

**Промежуточный вывод:** Лучший параметр эстиматоров RandomForest: 100, максимальной глубины: 10. F1 = 0.345, что не удовлетворяет заданному условию.

### Тестирование лучшей модели

Исходя из оценки метрики, остановимся на ***LogisticRegression***, как лучшей по F1. Проверим её качество на тестовой выборке.

In [55]:
# Протестируем наилучшую модель на тестовой выборке.
predictions = grid_lr.best_estimator_.predict(X_test)
test_f1 = f1_score(y_test, predictions)

print('F1 для лучшей модели LogisticRegression:', test_f1)

F1 для лучшей модели LogisticRegression: 0.7809655172413794


**Промежуточный вывод:** F1 для лучшей модели LogisticRegression на тестовой выборке = 0.781, что удовлетворяет заданному условию.

## Выводы

**Цель исследования:** Обучили модели классифицировать комментарии на позитивные и негативные. Оценили их по критериям, указанным заказчиком.

**Подготовка данных:** Проверили данные на пропуски. Очистили тексты комментариев от разделителей и заглавных символов и лемматизировали их.

**Обучение моделей:** Разделили данные на обучающую и тестовую выборки. Создали пайплайн, в котором векторизировали признаки с использованием стоп-слов. Обучили 3 модели машинного обучения, подобрав гиперпараметры, оценили модели по метрике F1, которая по указаниям заказчика должна быть не меньше 0.75:
- LogisticRegression: Лучший параметр C логистической регрессии: 10. F1 = 0.775, что удовлетворяет заданному условию.
- DecisionTreeClassifier: Лучший параметр максимальной глубины древа решений: 20. F1 = 0.64, что не удовлетворяет заданному условию.
- RandomForestClassifier: Лучший параметр эстиматоров RandomForest: 100, максимальной глубины: 10. F1 = 0.345, что не удовлетворяет заданному условию.

Исходя из оценки метрики, остановились на LogisticRegression, как лучшей по F1. F1 для лучшей модели LogisticRegression на тестовой выборке = 0.781, что удовлетворяет заданному условию.